In [1]:
# -*- coding: utf-8 -*-

import os
import shutil
import subprocess

import speech_recognition as sr

# 音声ファイルの分割
import wave
import math
import struct
import numpy as geek
import time
#from scipy import fromstring, int16 

def meke_c():
    
    os.makedirs('./mp4', exist_ok=True)
    
    os.makedirs('./wav', exist_ok=True)
    
    os.makedirs('./cut_wav',exist_ok=True)

    os.makedirs('./out_text',exist_ok=True)

    return

def meke_d():
    

    shutil.rmtree('./mp4')
    shutil.rmtree('./wav')
    shutil.rmtree('./cut_wav')

    return
    

def mov_to_mp4(movf):
       
    mp4f = movf.replace('.MOV', '.mp4')
    mp4f = mp4f.replace('./mov/', './mp4/')
   
    if os.path.exists(mp4f):
     os.remove(mp4f)

#    subprocess.run(['ffmpeg', '-i', movf, mp4f], 
#                   encoding='utf-8', stdout=subprocess.PIPE)
    
    p = subprocess.Popen(['ffmpeg', '-i', movf, mp4f], 
                   encoding='utf-8', stdout=subprocess.PIPE)
    p.wait()
    
    return mp4f

# mp4から音声ファイルへの変換
def mp4_to_wav(mp4f):
    
    work_wav = mp4f.replace('.mp4', '.wav')
    work_wav = work_wav.replace('./mp4/', './wav/')
    
    if os.path.exists(work_wav):
     os.remove(work_wav)

#    subprocess.run(['ffmpeg', '-i', mp4f, work_wav], 
#                   encoding='utf-8', stdout=subprocess.PIPE)
    
    p = subprocess.Popen(['ffmpeg', '-i', mp4f, work_wav], 
                   encoding='utf-8', stdout=subprocess.PIPE)
    p.wait()
    print("mp4から音声ファイルへの変換")
    print(work_wav)
    return work_wav

# 音声ファイルの分割(デフォルト30秒)
def cut_wav(wavf,time=30):
    # timeの単位は[sec]
    # ファイルを読み出し
    wr = wave.open(wavf, 'r')

    # waveファイルが持つ性質を取得
    ch = wr.getnchannels()
    width = wr.getsampwidth()
    fr = wr.getframerate()
    fn = wr.getnframes()
    total_time = 1.0 * fn / fr
    integer = math.floor(total_time) # 小数点以下切り捨て
    t = int(time)  # 秒数[sec]
    frames = int(ch * fr * t)
    num_cut = int(integer//t)

    # waveの実データを取得し、数値化
    data = wr.readframes(wr.getnframes())
    wr.close()
    X = geek.fromstring(data, dtype='int16')
  
    # wavファイルを削除
    os.remove(wavf)
  
    outf_list = []
    for i in range(num_cut):
        # 出力データを生成
        outf = './cut_wav/' + str(i).zfill(3) + '.wav'
        start_cut = i*frames
        end_cut = i*frames + frames
        Y = X[start_cut:end_cut]
        outd = struct.pack("h" * len(Y), *Y)

        # 書き出し
        ww = wave.open(outf, 'w')
        ww.setnchannels(ch)
        ww.setsampwidth(width)
        ww.setframerate(fr)
        ww.writeframes(outd)
        ww.close()
      
        # リストに追加
        outf_list.append(outf)
        
    print(outf_list)
    return outf_list

# 複数ファイルの音声のテキスト変換
def cut_wavs_str(outf_list):
    output_text = ''
    cont = 0
    # 複数処理
    print('音声のテキスト変換')
    for fwav in outf_list:
            try:
                r = sr.Recognizer()
                # 音声->テキスト
                with sr.AudioFile(fwav) as source:
                    print('音声→テキスト ')
                    audio = r.record(source)
                    text = r.recognize_google(audio, language='ja-JP')
                    # 各ファイルの出力結果の結合
                    output_text = output_text + text + '\n'
                    # wavファイルを削除
                    os.remove(fwav)
            except sr.UnknownValueError:
                print("could not understand audio" + fwav)
                # wavファイルを削除
                os.remove(fwav)
            except sr.RequestError as e:
                print("Could not request results from Google Speech Recognition service; {0}".format(e))
    return output_text


#main関数的な。。
path = './mov/'

meke_c()
#    check_directory()
  
#  for movf in os.listdir(path):
#movf = path + "サムネ.MOV"

outf_list = []

for movf in os.listdir(path):
    base,ext = os.path.splitext(movf)

    if ext == '.MOV':
        outf_list.append(path + base + '.MOV')
#outf_list.append('./mov/トーク２.MOV')

print(outf_list)

for movf in outf_list:

    print(movf)
    # 動画ファイルの変換  
    mp4f = mov_to_mp4(movf)

    print(mp4f)

    # 音声ファイルへの変換
    wav_file = mp4_to_wav(mp4f)

    print(wav_file)

    # 音声ファイルの分割(デフォルト30秒)
    cut_wavs = cut_wav(wav_file)

    # 複数ファイルの音声のテキスト変換
    out_text = cut_wavs_str(cut_wavs)

    # テキストファイルへの入力
    mp4f_name = os.path.basename(mp4f)
    txt_file = './out_text/' + mp4f_name.replace('.mp4', '.txt')
    print('テキスト出力')
    print(txt_file)
    f = open(txt_file, 'w')
    f.write(out_text)
    f.close()

print("終了　後処理")
meke_d()



['./mov/塩焼き５.MOV', './mov/塩焼き９.MOV', './mov/塩焼き６.MOV', './mov/塩焼き８.MOV', './mov/塩焼き２.MOV', './mov/塩焼き３.MOV', './mov/トーク最後.MOV', './mov/サムネ.MOV', './mov/トーク１.MOV', './mov/塩焼き１２.MOV', './mov/塩焼き１.MOV', './mov/塩１.MOV', './mov/塩焼き１１.MOV', './mov/塩焼き４.MOV', './mov/塩焼き１０.MOV', './mov/トーク２.MOV', './mov/塩焼き７.MOV', './mov/塩２.MOV']
./mov/塩焼き５.MOV


ffmpeg version 4.3.4-0+deb11u1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 10 (Debian 10.2.1-6)
  configuration: --prefix=/usr --extra-version=0+deb11u1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-avresample --disable-filter=resample --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librsvg --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab 

./mp4/塩焼き５.mp4
mp4から音声ファイルへの変換
./wav/塩焼き５.wav
./wav/塩焼き５.wav
[]
音声のテキスト変換
テキスト出力
./out_text/塩焼き５.txt
./mov/塩焼き９.MOV


[libx264 @ 0x55a6a383e500] using cpu capabilities: MMX2 SSE2Fast SSSE3 SSE4.2 AVX FMA3 BMI2 AVX2
[libx264 @ 0x55a6a383e500] profile High, level 4.0, 4:2:0, 8-bit
[libx264 @ 0x55a6a383e500] 264 - core 160 r3011 cde9a93 - H.264/MPEG-4 AVC codec - Copyleft 2003-2020 - http://www.videolan.org/x264.html - options: cabac=1 ref=3 deblock=1:0:0 analyse=0x3:0x113 me=hex subme=7 psy=1 psy_rd=1.00:0.00 mixed_ref=1 me_range=16 chroma_me=1 trellis=1 8x8dct=1 cqm=0 deadzone=21,11 fast_pskip=1 chroma_qp_offset=-2 threads=18 lookahead_threads=3 sliced_threads=0 nr=0 decimate=1 interlaced=0 bluray_compat=0 constrained_intra=0 bframes=3 b_pyramid=2 b_adapt=1 b_bias=0 direct=1 weightb=1 open_gop=0 weightp=2 keyint=250 keyint_min=25 scenecut=40 intra_refresh=0 rc_lookahead=40 rc=crf mbtree=1 crf=23.0 qcomp=0.60 qpmin=0 qpmax=69 qpstep=4 ip_ratio=1.40 aq=1:1.00
Output #0, mp4, to './mp4/塩焼き９.mp4':
  Metadata:
    major_brand     : qt  
    minor_version   : 0
    compatible_brands: qt  
    com.apple.photo

./mp4/塩焼き９.mp4
mp4から音声ファイルへの変換
./wav/塩焼き９.wav
./wav/塩焼き９.wav
[]
音声のテキスト変換
テキスト出力
./out_text/塩焼き９.txt
./mov/塩焼き６.MOV


...

mp4から音声ファイルへの変換
./wav/トーク２.wav
./wav/トーク２.wav
['./cut_wav/000.wav', './cut_wav/001.wav', './cut_wav/002.wav', './cut_wav/003.wav', './cut_wav/004.wav', './cut_wav/005.wav', './cut_wav/006.wav', './cut_wav/007.wav', './cut_wav/008.wav', './cut_wav/009.wav', './cut_wav/010.wav', './cut_wav/011.wav', './cut_wav/012.wav', './cut_wav/013.wav', './cut_wav/014.wav', './cut_wav/015.wav', './cut_wav/016.wav']
音声のテキスト変換
音声→テキスト 
音声→テキスト 
音声→テキスト 
音声→テキスト 
could not understand audio./cut_wav/003.wav
音声→テキスト 
音声→テキスト 
音声→テキスト 
音声→テキスト 
音声→テキスト 
音声→テキスト 
音声→テキスト 
音声→テキスト 
音声→テキスト 
音声→テキスト 
音声→テキスト 
音声→テキスト 
音声→テキスト 
テキスト出力
./out_text/トーク２.txt
./mov/塩焼き７.MOV


ffmpeg version 4.3.4-0+deb11u1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 10 (Debian 10.2.1-6)
  configuration: --prefix=/usr --extra-version=0+deb11u1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-avresample --disable-filter=resample --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librsvg --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab 

./mp4/塩焼き７.mp4
mp4から音声ファイルへの変換
./wav/塩焼き７.wav
./wav/塩焼き７.wav
['./cut_wav/000.wav']
音声のテキスト変換
音声→テキスト 
テキスト出力
./out_text/塩焼き７.txt
./mov/塩２.MOV


ffmpeg version 4.3.4-0+deb11u1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 10 (Debian 10.2.1-6)
  configuration: --prefix=/usr --extra-version=0+deb11u1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-avresample --disable-filter=resample --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librsvg --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab 

./mp4/塩２.mp4
mp4から音声ファイルへの変換
./wav/塩２.wav
./wav/塩２.wav
['./cut_wav/000.wav']
音声のテキスト変換
音声→テキスト 
テキスト出力
./out_text/塩２.txt
終了　後処理
